In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
# req_311 = pd.read_csv("/home/ec2-user/data/311_service_requests.csv", low_memory=False)
# comm_health = pd.read_csv("/home/ec2-user/data/community_health.csv", low_memory=False)
# demo = pd.read_csv("/home/ec2-user/data/demographics.csv", low_memory=False)
# demo_city = pd.read_csv("/home/ec2-user/data/demographics_city.csv", low_memory=False)
# env_rad = pd.read_csv("/home/ec2-user/data/environmental_radiation.csv", low_memory=False)
# geographic = pd.read_csv("/home/ec2-user/data/geographic.csv", low_memory=False)
# inspections = pd.read_csv("/home/ec2-user/data/food_establishment_inspections.csv", \
#                           encoding='latin1', engine='python')
# venues = pd.read_csv("/home/ec2-user/data/food_venues.csv", low_memory=False, encoding='latin1')

In [3]:
demo_2013 = pd.read_excel("/home/ec2-user/citadel2018datathon/demo_data/dem_2013acs5yr_nta.xlsx")
demo_2013 = demo_2013[["GeoID", "PopE"]]
hous_2013 = pd.read_excel("/home/ec2-user/citadel2018datathon/demo_data/hsg_2013acs5yr_nta.xlsx")
hous_2013 = hous_2013[["GeoID", "MdVlE"]]
soc_2013 = pd.read_excel("/home/ec2-user/citadel2018datathon/demo_data/soc_2013acs5yr_nta.xlsx")
soc_2013 = soc_2013[["GeoID", "EA_BchDE"]]
econ_2013 = pd.read_excel("/home/ec2-user/citadel2018datathon/demo_data/eco_2013acs5yr_nta.xlsx")
econ_2013 = econ_2013[["GeoID", "MdHHIncE"]]

In [4]:
master2013 = pd.concat([demo_2013["GeoID"],demo_2013["PopE"],hous_2013["MdVlE"], \
                        soc_2013["EA_BchDE"],econ_2013["MdHHIncE"]],axis = 1)

In [5]:
demo_2016 = pd.read_excel("/home/ec2-user/citadel2018datathon/demo_data/demo_2016acs5yr_nta.xlsx")
demo_2016 = demo_2016[["GeoID", "Pop_1E"]]
hous_2016 = pd.read_excel("/home/ec2-user/citadel2018datathon/demo_data/hous_2016acs5yr_nta.xlsx")
hous_2016 = hous_2016[["GeoID", "MdVlE"]]
soc_2016 = pd.read_excel("/home/ec2-user/citadel2018datathon/demo_data/soc_2016acs5yr_nta.xlsx")
soc_2016 = soc_2016[["GeoID", "EA_BchDE"]]
econ_2016 = pd.read_excel("/home/ec2-user/citadel2018datathon/demo_data/econ_2016acs5yr_nta.xlsx")
econ_2016 = econ_2016[["GeoID", "MdHHIncE"]]

In [6]:
master2016 = pd.concat([demo_2016["GeoID"],demo_2016["Pop_1E"],hous_2016["MdVlE"], \
                        soc_2016["EA_BchDE"],econ_2016["MdHHIncE"]],axis = 1)

In [7]:
master = pd.concat([master2013, master2016[["Pop_1E", "MdVlE", "EA_BchDE", "MdHHIncE"]]], axis = 1)
master.columns = ["NTAID", "2013_Pop", "2013_Median_Home_Value", "2013_Pct_Bachelors", "2013_Median_Income", \
                 "2016_Pop", "2016_Median_Home_Value", "2016_Pct_Bachelors", "2016_Median_Income"]

In [8]:
# solving 40th percentile median income 2013:
med_inc_2013 = master["2013_Median_Income"].dropna().astype('int64').sort_values()
n = len(med_inc_2013)
pctile = math.ceil(0.4*n)
thresh = med_inc_2013.iloc[pctile]
less_than_40th_medincome = master["2013_Median_Income"] < thresh

In [9]:
# same for median home value
med_house_2013 = master["2013_Median_Home_Value"].dropna().astype(str).str.replace(",","").astype('int64').sort_values()
n = len(med_house_2013)
pctile = math.ceil(0.4*n)
thresh = med_house_2013.iloc[pctile]
less_than_40th_medhomevalue = master["2013_Median_Home_Value"].astype(str).str.replace(",","").astype('float') < thresh

In [10]:
# pct increase education
pct_bach_2013 = master["2013_Pct_Bachelors"]
pct_bach_2016 = master["2016_Pct_Bachelors"]
change = pct_bach_2016 - pct_bach_2013
clean_change = change.dropna().astype('float').sort_values(ascending=False)
n = len(change)
pctile = math.floor(0.33*n)
thresh = clean_change.iloc[pctile]
top_3rd_education = change > thresh

In [11]:
# percentage increase in median home value
hv2013 = master["2013_Median_Home_Value"].astype(str).str.replace(",","").astype('float')
hv2016 = master["2016_Median_Home_Value"].astype(str).str.replace(",","").astype('float')
pct_change = (hv2016-hv2013)/hv2013
clean_pct_change = pct_change.dropna().astype('float').sort_values(ascending=False)
n = len(pct_change)
pctile = math.floor(0.33*n)
thresh = clean_pct_change.iloc[pctile]
top_3rd_home_value_pct_increase = pct_change > thresh

In [12]:
# median home value increased
home_value_increased = hv2016-hv2013 > 0

In [13]:
Y_getter = pd.concat([less_than_40th_medincome, less_than_40th_medhomevalue, top_3rd_education, \
                      top_3rd_home_value_pct_increase, home_value_increased], axis = 1)
Y_getter.columns = ["less_than_40th_medincome", "less_than_40th_medhomevalue", "top_3rd_education", \
                      "top_3rd_home_value_pct_increase", "home_value_increased"]

In [14]:
label = Y_getter.all(axis = 1).astype('int64')
Y = pd.concat([master["NTAID"], label],axis = 1)
Y.columns = ["NTAID", "Y"]
Y.to_csv("/home/ec2-user/data/Y.csv")

In [15]:
ntaids = []
for index, ntaid, income, home, education, homepct, homediff in pd.concat(Y_getter).itertuples():
    if income and home and education and homepct and homediff:
        ntaids.append(ntaid)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [16]:
Y[Y["Y"] == 1].to_csv("/home/ec2-user/data/gentrified.csv")